<html><font face="Courier New"><head><h1><b>Projeto 2<br>Aprendizado de máquina: reconhecimento de números</b></h1></head><br>
<body>
<h1>Introdução</h1><br>
<font size="3">
A grosso modo, Machine Learning, ou aprendizagem de máquina, é uma importante área da inteligência artificial onde é possível criar algoritmos para ensinar uma máquina a desempenhar tarefas.

Pega-se um conjunto de dados de entrada e com base em determinados padrões encontrados gera-se saídas das quais podem ser usadas para reconhecimento de números, caracteres, imagens, reconhecimento facial, etc...

<h1>Conceitos</h1><br>
As redes neurais artificiais são baseadas nas tentativas de simular o sistema nervoso humano no que se refere a sua capacidade de aprender e de se adaptar, em outras palavras, uma simulação muito simplificada da operação do cérebro humano. Estas redes neurais artificiais são constituídas de neurônios artificiais (Figura 1).

<img src="https://github.com/wars2/mnist-intropython/blob/reconhecimento/Imagens/neuron.png" align="center"></img>

<font size="2"><center>Figura 1 - Diagramação de um neurônio artificial.</center></font>

A estrutura de um neurônio pode ser representada como uma composição das seguintes unidades:
* Entradas: <b>Xi,...,Xn</b>;

* Pesos: <b>Wi,...,Wn</b>;

* Função de transferência: <b>$\sum$</b>;

* Entrada de rede de um neurônio: <b>NET</b>;

* Função de ativação: <b>F(x)</b>;

* Saída: <b>Out.</b>


</font></body></font></html>